In [1]:
import graphlab

In [109]:
movie_data = graphlab.SFrame('movie__data.csv')

Finished parsing file /home/vikash/movie__data.csv

Parsing completed. Parsed 100 lines in 15.0123 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 41643 lines. Lines per second: 5914.49

Finished parsing file /home/vikash/movie__data.csv

Parsing completed. Parsed 50000 lines in 7.26237 secs.

In [3]:
important_words = ['love','great','excellent','outstanding','phenomenal','fantastic','awesome','like','better','good',
                   'favourable','marvellous','brilliant','trailer',
                   'movie','cast','quality','actors','actress','sound','image','blockbuster','art','direction','awful','dissapointed','crazy','waste','time','money','bad',
                   'worst','not','neither','hollywood','india','celebrity','nonsense','disrespectful','annoying','camera','acting','talent','horrible','despicable','offensive',
                   'pessimistic','ruthless','prejudice','angry','dirty','fail','flop','hit','rude','underrated','sick','stupid','hate','hell','adorable','genius','lovely','admire',
                   'beautiful','nice','pity','natural','bliss','pretty','rate','rating','star','hall','innovative','fun','funny','classic','perfect','knowledge','motivating',
                   'cinematic','cinema','songs','music','handsome','laugh','terrific','jealous','imperfect','ill','insane','boring','reject','poor',
                  'cheap','enough','make','almost','scary','sorry','no','cried','upset','sad','awkward','weird','kind','effective','action','wonderful','cool','cute','pretend',
                   'positive','negative','success','failure','merit','drama','commedy','worth','right','satisfied','disagree','theme','easy','talented','bravo','happy','joy',
                   'idea','stupendous','vital','moving','imagine','episode','evidence','murder','learning','characters','percepion','judgement','really','loopholes','performance',
                   'comments','different','romance','enjoyable','silhouette','except','film','entertaining','overrated','director','creepy','eerie','phone','']
            
            

In [108]:
len(important_words)

197

In [110]:
for word in important_words:
    movie_data[word] = movie_data['review'].apply(lambda s : s.split().count(word))

In [114]:
movie_data['jennifer'].sum()

533

In [7]:
import numpy as np

In [8]:
def get_numpy_data(data_sframe, features, label):
    data_sframe['intercept'] = 1
    features = ['intercept'] + features
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    label_sarray = data_sframe[label]
    label_array = label_sarray.to_numpy()
    return(feature_matrix, label_array)

In [115]:
feature_matrix, sentiment = get_numpy_data(movie_data, important_words, 'sentiment') 

In [10]:
import math
def predict_probability(feature_matrix, coefficients):
    #dot product of feature_matrix and coefficients
    dot_product = np.dot(feature_matrix,coefficients)
    # P(y_i = +1 | x_i, w) using the link function
    predictions = 1/(1+np.exp(-dot_product))
    # return predictions
    return predictions

In [11]:
def feature_derivative(errors, feature):     
    # dot product of errors and feature
    derivative = np.dot(errors,feature)
    # Return the derivative
    return derivative

In [12]:
from math import sqrt

def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in xrange(max_iter):
        # P(y_i = +1|x_i,w) using your predict_probability() function
        predictions = predict_probability(feature_matrix,coefficients) 
        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1) 
        # Compute the errors as indicator - predictions
        errors = indicator - predictions
        for j in xrange(len(coefficients)): # loop over each coefficient
            derivative = feature_derivative(errors,feature_matrix[:,j])   
            # add the step size times the derivative to the current coefficient
            coefficients[j] = coefficients[j] + step_size*derivative
    return coefficients

In [116]:
coefficients = logistic_regression(feature_matrix, sentiment, initial_coefficients=np.zeros(len(important_words)+1),
                                   step_size=1e-7, max_iter=301)

In [117]:
# Compute the scores as a dot product between feature_matrix and coefficients.
scores = np.dot(feature_matrix, coefficients)

In [118]:
class_predictions = np.array(graphlab.SArray(scores).apply(lambda x: 1 if x>0 else -1))

In [119]:
unique,count = np.unique(class_predictions,return_counts=True)
print unique,count

[-1  1] [28563 21437]


In [120]:
(class_predictions == movie_data['sentiment']).sum()

15232

In [82]:
movie_data.shape

(50000, 159)

In [18]:
m_d = graphlab.SFrame('movie__data.csv')

Finished parsing file /home/vikash/movie__data.csv

Parsing completed. Parsed 100 lines in 3.38143 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 41643 lines. Lines per second: 19674.8

Finished parsing file /home/vikash/movie__data.csv

Parsing completed. Parsed 50000 lines in 2.33328 secs.

In [19]:
m_d['word_count'] = graphlab.text_analytics.count_words(m_d['review'])

In [20]:
train,test = m_d.random_split(.8)

In [21]:
model = graphlab.logistic_classifier.create(train,features=['word_count'],target='sentiment',l1_penalty=1)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 38103

...

| 1         | 2        | 0.000008  | 16.974000    | 0.920557          | 0.863659            |

| 2         | 3        | 0.000008  | 18.411970    | 0.960528          | 0.885969            |

| 3         | 4        | 0.000002  | 21.516338    | 0.967010          | 0.900843            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [22]:
model.evaluate(train)['accuracy']

0.9636839481555334

In [23]:
model.evaluate(test)['accuracy']

0.887246963562753

In [120]:
model = graphlab.boosted_trees_classifier.create(movie_data,features=important_words,target='sentiment')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: Detected extremely low variance for feature(s) 'percepion' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 47569

...

| 4         | 1.024860     | 0.738822          | 0.725216            | 0.556417          | 0.562508            |

| 5         | 1.264410     | 0.741786          | 0.724805            | 0.542434          | 0.549452            |

| 6         | 1.513438     | 0.745401          | 0.729741            | 0.530560          | 0.540010            |

| 10        | 2.458434     | 0.760684          | 0.746195            | 0.501183          | 0.516147            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [105]:
model.evaluate(movie_data)['accuracy']

0.81954

In [30]:
sf = graphlab.SFrame({'review':['I did not like the movie at all','I love this movie','Awful movie ever','Cast was not that good and movie was bad too','Actors were good and their acting was great','Overrated movie, is not up to the mark']})

In [95]:
sf = ({'review':['I love this movie','Awful movie ever','Cast was not that good and movie was bad too','Actors were good and their acting was great','Overrated movie, is not up to the mark']})

In [100]:
for word in important_words:
    sf[word] = sf['review'].apply(lambda s : s.split().count(word))

In [107]:
model.predict(sf)

dtype: int
Rows: 5
[1, 1, 1, 1, 1]

In [111]:
gmodel = graphlab.logistic_classifier.create(m_d,features=['word_count'],target='sentiment')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 47525

...

| 9         | 13       | 1.000000  | 10.593825    | 0.993435          | 0.897778            |

| 10        | 14       | 1.000000  | 11.558527    | 0.997012          | 0.888485            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [113]:
gmodel.coefficients.tail()

name,index,class,value,stderr
word_count,samoa,1,0.639701163444,None
word_count,vaitongi,1,0.639701163444,None
word_count,polymath,1,0.639701163444,None
word_count,heuristic,1,0.639701163444,None
word_count,hominids,1,0.639701163444,None
word_count,farraginous,1,0.639701163444,None
word_count,maclaughlin,1,0.319850581722,None
word_count,tannen,1,0.639701163444,None
word_count,joists,1,0.639701163444,None
word_count,darwinianed,1,0.639701163444,None


In [116]:
model.coefficients

AttributeError: 'BoostedTreesClassifier' object has no attribute 'coefficients'

In [121]:
model.predict(sf)

dtype: int
Rows: 5
[1, 1, 1, 1, 1]

In [123]:
m_d['tf_idf'] = graphlab.text_analytics.tf_idf(m_d['review'])

In [129]:
tfmodel = graphlab.logistic_classifier.create(m_d,features=['tf_idf'],target='sentiment',verbose=False,l1_penalty=2)

In [130]:
tfmodel.evaluate(m_d)['accuracy']

0.96526

In [131]:
tfmodel.predict(sf)

dtype: int
Rows: 5
[1, 1, 1, 1, 1]

In [148]:
df = graphlab.SFrame({'review':['I love this movie','Awful movie ever','Cast was not that good and movie was bad too','Actors were good and their acting was great','Overrated movie, is not up to the mark','i did not like the movie','waste of money','Lovely movie and our kids enjoyed it too']})

In [149]:
df['tf_idf'] = graphlab.text_analytics.tf_idf(df['review'])

In [153]:
tfmodel.predict(df,output_type='probability')

dtype: float
Rows: 8
[0.502454397566931, 0.47100587716158865, 0.4863678019352587, 0.5207111418826734, 0.49845397540230557, 0.4958164985061376, 0.45981560382361836, 0.556684909985257]

In [155]:
df['tf_idf'][0]

{'i': 1.3862943611198906,
 'love': 2.0794415416798357,
 'movie': 0.47000362924573563,
 'this': 2.0794415416798357}

In [181]:
new = graphlab.SFrame({'review':['So bad movie']})

In [182]:
new['tf_idf'] = graphlab.text_analytics.tf_idf(new['review'])

In [183]:
tfmodel.predict(new,output_type='probability')

dtype: float
Rows: 1
[0.5016258710331564]

In [176]:
graphlab.logistic_classifier

review,tf_idf
What an incredible movietoo liked every part of ...,"{'liked': 0.0, 'just':0.6931471805599453, ' ..."
liked the movie so muchsongs were good too ...,"{'liked': 0.0, 'good':0.0, 'movie': 0.0, ..."


In [177]:
new['tf_idf']

dtype: dict
Rows: 2
[{'liked': 0.0, 'just': 0.6931471805599453, 'it': 0.6931471805599453, 'an': 0.6931471805599453, 'marvelous': 0.6931471805599453, 'in': 0.6931471805599453, 'what': 0.6931471805599453, 'dancing': 0.6931471805599453, 'movie': 0.0, 'literaly': 0.6931471805599453, 'too': 0.0, 'between': 0.6931471805599453, 'good': 0.0, 'that': 0.6931471805599453, 'started': 0.6931471805599453, 'part': 0.6931471805599453, 'every': 0.6931471805599453, 'i': 0.6931471805599453, 'of': 0.6931471805599453, 'incredible': 0.6931471805599453, 'so': 0.0, 'were': 0.0, 'songs': 0.0}, {'liked': 0.0, 'good': 0.0, 'movie': 0.0, 'much': 0.6931471805599453, 'so': 0.0, 'too': 0.0, 'were': 0.0, 'the': 0.6931471805599453, 'songs': 0.0}]

In [185]:
m_d['1grams_feature'] = graphlab.text_analytics.count_ngrams(m_d['review'],1)
m_d['2grams_feature'] = graphlab.text_analytics.count_ngrams(m_d['review'],2)

In [186]:
m_d

review,sentiment,word_count,tf_idf
ice the limerick a viruspulled out of the ice ...,1,"{'all': 6, 'whoever': 1,'caused': 1, 'focus': 1, ...","{'all':3.9480703427501433, ..."
big disappointment clashby night is much to t ...,0,"{'true': 1, 'over': 1,'years': 1, 'talky': 1, ...","{'true':2.557993604895531, ..."
i really wish that whenmaking a comedy the ...,0,"{'this': 2, 'people': 1,'is': 2, 'tootsie': 1, ...","{'this':0.1980059796840157, ..."
i absolutely loved thismovie it was so good ...,1,"{'rating': 1, 'all': 1,'help': 1, 'ending': 1, ...","{'rating':3.4370318346713713, ..."
i passed this one on theshelf a few times loo ...,0,"{'gold': 1, 'over': 1,'lack': 1, 'front': 1, ...","{'gold':4.601178164718554, ..."
i see a lot of reallynegative posts by people ...,1,"{'all': 2, 'just': 1,'september': 1, 'leads': ...","{'all':1.3160234475833812, ..."
i first saw this filmwhen i was around 6 or 7 ...,1,"{'all': 2,'particularly': 1, ...","{'all':1.3160234475833812, ..."
i used to enjoy happyever after but was ...,1,"{'heavily': 1, 'ended':1, 'radio': 2, ...","{'heavily':4.961845129926823, ..."
billy wilder is cocredited for the story ...,1,"{'and': 8, 'all': 2,'co': 1, 'cynical': 1, ...","{'and':0.2770628279555468, ..."
this is a gory mess andpretty convincing corbin ...,1,"{'control': 1, 'and': 3,'his': 2, 'performer' ...","{'control':4.040993386725106, 'a ..."


In [ ]:
gramsmodel = graphlab.classifier.create(m_d,features=['1grams_feature','2grams_feature'], target='sentiment')

In [31]:
sf['word_count'] = graphlab.text_analytics.count_words(sf['review'])

In [32]:
model.predict(sf)

dtype: int
Rows: 6
[0, 1, 0, 0, 1, 0]

In [33]:
sf

review,word_count
I did not like the movieat all ...,"{'all': 1, 'like': 1,'did': 1, 'movie': 1, ..."
I love this movie,"{'this': 1, 'movie': 1,'love': 1, 'i': 1} ..."
Awful movie ever,"{'awful': 1, 'movie': 1,'ever': 1} ..."
Cast was not that goodand movie was bad too ...,"{'and': 1, 'good': 1,'that': 1, 'movie': 1, ..."
Actors were good andtheir acting was great ...,"{'and': 1, 'great': 1,'good': 1, 'their': 1, ..."
"Overrated movie, is notup to the mark ...","{'is': 1, 'up': 1,'movie,': 1, 'mark': 1, ..."


In [29]:
model.save('my_model')

In [34]:
movie_data

review,sentiment,love,great,excellent,outstanding,phenomenal,fantastic,awesome
ice the limerick a viruspulled out of the ice ...,1,1,2,0,0,0,0,1
big disappointment clashby night is much to t ...,0,0,0,0,0,0,0,0
i really wish that whenmaking a comedy the ...,0,0,0,0,0,0,0,0
i absolutely loved thismovie it was so good ...,1,0,0,0,0,0,0,0
i passed this one on theshelf a few times loo ...,0,0,1,0,0,0,0,0
i see a lot of reallynegative posts by people ...,1,0,0,1,0,0,0,0
i first saw this filmwhen i was around 6 or 7 ...,1,0,0,0,0,0,2,0
i used to enjoy happyever after but was ...,1,0,1,0,0,0,0,0
billy wilder is cocredited for the story ...,1,0,1,0,0,0,0,0
this is a gory mess andpretty convincing corbin ...,1,0,0,0,0,0,0,0


In [44]:
sf = graphlab.SArray({'review':['My children loved the movie and they had a great weekend ','Songs was good but acting sucked','Beautiful actress but movie sucked to the core','I did not like the movie at all','I love this movie','Awful movie ever','Cast was not that good and movie was bad too','Actors were good and their acting was great','Overrated movie, is not up to the mark']})

TypeError: Unexpected data source. Possible data source types are: list, numpy.ndarray, pandas.Series, and string(url)

In [39]:
sf['word_count'] = graphlab.text_analytics.count_words(sf['review'])

In [40]:
model.predict(sf)

dtype: int
Rows: 9
[1, 0, 0, 0, 1, 0, 0, 1, 0]

In [41]:
load_model = graphlab.load_model('my_model')

In [42]:
load_model.predict(sf)

dtype: int
Rows: 9
[1, 0, 0, 0, 1, 0, 0, 1, 0]

In [45]:
sf['word_count']

dtype: dict
Rows: 9
[{'a': 1, 'and': 1, 'great': 1, 'loved': 1, 'weekend': 1, 'had': 1, 'they': 1, 'movie': 1, 'the': 1, 'my': 1, 'children': 1}, {'good': 1, 'but': 1, 'acting': 1, 'sucked': 1, 'was': 1, 'songs': 1}, {'beautiful': 1, 'core': 1, 'movie': 1, 'but': 1, 'to': 1, 'actress': 1, 'the': 1, 'sucked': 1}, {'all': 1, 'like': 1, 'did': 1, 'movie': 1, 'i': 1, 'at': 1, 'not': 1, 'the': 1}, {'this': 1, 'movie': 1, 'love': 1, 'i': 1}, {'awful': 1, 'movie': 1, 'ever': 1}, {'and': 1, 'good': 1, 'that': 1, 'movie': 1, 'cast': 1, 'bad': 1, 'too': 1, 'not': 1, 'was': 2}, {'and': 1, 'great': 1, 'good': 1, 'their': 1, 'acting': 1, 'actors': 1, 'were': 1, 'was': 1}, {'is': 1, 'up': 1, 'movie,': 1, 'mark': 1, 'to': 1, 'overrated': 1, 'not': 1, 'the': 1}]

In [46]:
model.predict(sf,output_type='probability')

dtype: float
Rows: 9
[0.5391021204840808, 0.4753080771745704, 0.49224178951907543, 0.4930174856941517, 0.5084854849002056, 0.465840498307592, 0.4852026630214624, 0.5010885121522415, 0.4760303293062787]

In [56]:
sf = graphlab.SFrame({'review':['My children loved the movie and they had a great weekend ','One of the most beautiful movie ever','Trailer was pretty good but the movie sucked in total','what a creepy movie. Everything just sucked']})

In [52]:
p = model.predict(sf,output_type='probability')

In [51]:
model.predict(sf)

dtype: int
Rows: 1
[1]

In [54]:
p[0]

0.5009738191717346

In [55]:
sf

review
My children loved themovie and they had a ...


In [57]:
for word in important_words:
    sf[word] = sf['review'].apply(lambda s : s.split().count(word))

In [58]:
sf['intercept'] = 1
features = ['intercept'] + important_words
features_sframe = sf[features]
f_m = features_sframe.to_numpy()

In [59]:
score_new = np.dot(f_m,coefficients)

In [62]:
predictions = np.array(graphlab.SArray(score_new).apply(lambda x: 1 if x>0 else 0))

In [63]:
predictions

array([1, 0, 0, 1])

In [64]:
predict_probability(f_m,coefficients)

array([ 0.52078248,  0.49993277,  0.4953194 ,  0.5054806 ])

In [74]:
model.coefficients.topk('value',10)

name,index,class,value,stderr
word_count,videozone,1,0.54602975502,None
word_count,8bit,1,0.52025029987,None
word_count,tendon,1,0.517340845019,None
word_count,sandbag,1,0.509272544162,None
word_count,revoew,1,0.509272544162,None
word_count,scribble,1,0.508911195518,None
word_count,stigmatas,1,0.508825102297,None
word_count,commentates,1,0.508240110217,None
word_count,desensitizing,1,0.508240110217,None
word_count,fosberg,1,0.502347136603,None


In [77]:
movie_data['tf_idf']

RuntimeError: Runtime Exception. Column name tf_idf does not exist.

In [79]:
m_d['tf_idf'] = graphlab.text_analytics.tf_idf(m_d['review'])

In [80]:
train,test = m_d.random_split(.8)

In [81]:
model_tfidf = graphlab.logistic_classifier.create(train,features=['tf_idf'], target='sentiment',l1_penalty=1)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 37942

...

| 1         | 2        | 0.000008  | 25.986102    | 0.919851          | 0.848871            |

| 2         | 3        | 0.000008  | 28.351659    | 0.955985          | 0.864082            |

| 3         | 4        | 0.000002  | 33.398888    | 0.969770          | 0.883219            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [105]:
model.coefficients[62522]

{'class': 1,
 'index': 'barlo',
 'name': 'word_count',
 'stderr': None,
 'value': -0.0984116152286584}

In [106]:
important_words = ['love','great','excellent','outstanding','phenomenal','fantastic','awesome','like','better','good',
                   'favourable','marvellous','brilliant','trailer',
                   'movie','cast','quality','actors','actress','sound','image','blockbuster','art','direction','awful','dissapointed','crazy','waste','time','money','bad',
                   'worst','not','neither','hollywood','india','celebrity','nonsense','disrespectful','annoying','camera','acting','talent','horrible','despicable','offensive',
                   'pessimistic','ruthless','prejudice','angry','dirty','fail','flop','hit','rude','underrated','sick','stupid','hate','hell','adorable','genius','lovely','admire',
                   'beautiful','nice','pity','natural','bliss','pretty','rate','rating','star','hall','innovative','fun','funny','classic','perfect','knowledge','motivating',
                   'cinematic','cinema','songs','music','handsome','laugh','terrific','jealous','imperfect','ill','insane','boring','reject','poor',
                  'cheap','enough','make','almost','scary','sorry','no','cried','upset','sad','awkward','weird','kind','effective','action','wonderful','cool','cute','pretend',
                   'positive','negative','success','failure','merit','drama','commedy','worth','right','satisfied','disagree','theme','easy','talented','bravo','happy','joy',
                   'idea','stupendous','vital','moving','imagine','episode','evidence','murder','learning','characters','percepion','judgement','really','loopholes','performance',
                   'comments','different','romance','enjoyable','silhouette','except','film','entertaining','overrated','director','creepy','eerie','phone','frustrating','yet','overwhelming'
                  'joy','happiness','sadness','baby','freak','results','monseur','respect','streamwood','smooth','cumberbatch','cruise','jolie','pitt','brad','mexicanos','indians','americans'
                  'mission','impossible','possible','leonardo','johnny','hanks','smith','black','white','jennifer','dwayne','robert','samuel','proofread','hiatus','break','continue','mobile',
                  'machine']
            
            

In [107]:
len(important_words)

197

In [121]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix / norms
    return (normalized_features, norms)

In [122]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_outcome(feature_matrix,weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i] * ((output - prediction) + (weights[i]*feature_matrix[:,i]))).sum()

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + (l1_penalty/2)
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - (l1_penalty/2)
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [123]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    difference = np.array(initial_weights)*0.0
    found = 0
    weights = np.array(initial_weights)
    D = feature_matrix.shape[1]
    while(found == 0):
        for i in range(D):
            old_weights_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i,feature_matrix, output, weights,l1_penalty)
            difference[i] = np.abs(old_weights_i - weights[i])
        if(max(difference) < tolerance):
            return weights
            found = 1
    return weights

In [124]:
(class_predictions == sentiment).sum()

15232

In [125]:
15232/50000

0